## Посимвольная языковая модель. Часть 2.

В этом задании Вам нужно использовать написанную языковую модель для генерации воображаемых какслов.

**В процессе написания Вам нужно решить следующие проблемы**:
    
* как запоминать изменившееся состояние модели и передавать его на следующий временной шаг.
* что будет начальным состоянием модели.
* как понять, что слово закончилось и нужно прекратить генерацию.

**Результаты**:

* генератор слов на основе обученной посимвольной модели
* посимвольные вероятности сгенерированных слов

**Дополнительно**:

* ускорение модели за счёт побатчевой генерации

In [1]:
# it is better to do all imports at the first cell
import torch
from torch.nn import Module
from torch.nn import Embedding, RNN, GRU, LSTM, Linear
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np

import matplotlib.pyplot as plt
from re import findall

In [2]:
class CharRnnModel(Module):
    
    def __init__(self, hidden_size, emb_dim, vocab_len, num_layers=1, dropout=0.5):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.embed = Embedding(vocab_len, emb_dim)
        self.gru = GRU(emb_dim, hidden_size, num_layers=num_layers, dropout=dropout, batch_first=True)
        self.fc = Linear(hidden_size, vocab_len)
        
    def forward(self, inputs, h0=None):
        if len(inputs.shape) == 0:
            inputs = inputs.unsqueeze(0).unsqueeze(0)
        if len(inputs.shape) == 1:
            inputs = inputs.unsqueeze(1)
#         print(inputs.shape)
        emb = self.embed(inputs)
#         print(emb.shape)
        if h0 is None:
            gru_out, gru_hidden = self.gru(emb)
        else:
            gru_out, gru_hidden = self.gru(emb, h0)
#         print(gru_out.shape, gru_hidden.shape)
        out = self.fc(gru_out)
#         print(out.shape)
        return out, gru_hidden
    
    def init_hidden(self, batch_size):
        return torch.zeros(self.num_layers, batch_size, self.hidden_size)

In [3]:
t = torch.load('charRNNmodel.pt')
model_state_dict, model_params, vocab_data = t['model_state_dict'], t['model_params'], t['vocab_info']

In [4]:
print(model_params.keys())
print(vocab_data.keys())

dict_keys(['n_hidden', 'emb_dims', 'n_layers', 'dropout'])
dict_keys(['vocab', 's2i', 'i2s', 'spec_tokens'])


In [5]:
N_HIDDEN = model_params['n_hidden']
EMB_DIM = model_params['emb_dims']
N_LAYERS = model_params['n_layers']
VOCAB_LEN = len(vocab_data['vocab'])

model = CharRnnModel(N_HIDDEN, EMB_DIM, VOCAB_LEN, N_LAYERS)
model.load_state_dict(model_state_dict)

In [6]:
print(model)

CharRnnModel(
  (embed): Embedding(37, 200)
  (gru): GRU(200, 256, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=256, out_features=37, bias=True)
)


CharRnnModel(
  (embed): Embedding(37, 200)
  (gru): GRU(200, 256, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=256, out_features=37, bias=True)
)

In [7]:
h0 = model.init_hidden(1)
char = torch.tensor(vocab_data['s2i']['<'])
print(char.shape)
pred, h0 = model(char, h0)
print(torch.softmax(pred, dim=2))

torch.Size([])
tensor([[[1.3285e-04, 2.0944e-05, 2.5585e-05, 8.9748e-04, 4.6651e-02,
          5.2238e-02, 7.4105e-02, 3.5158e-02, 3.3890e-02, 2.4888e-03,
          8.2025e-03, 3.7570e-02, 3.2877e-02, 3.1101e-04, 5.9917e-02,
          3.0199e-02, 3.7426e-02, 4.8148e-02, 7.4061e-02, 1.3043e-01,
          4.1571e-02, 1.0073e-01, 4.2605e-02, 3.8967e-02, 7.9501e-03,
          1.3125e-02, 3.7002e-03, 1.2676e-02, 1.4796e-02, 2.9172e-03,
          5.4417e-05, 1.3052e-04, 2.1286e-04, 8.3352e-03, 2.3443e-03,
          3.5071e-03, 1.6301e-03]]], grad_fn=<SoftmaxBackward>)


In [8]:
def predict_next(char, h, k):
    assert(char in vocab_data['s2i'])
    
    char = torch.tensor(vocab_data['s2i'][char])
    pred, h = model(char, h)
    probs = torch.softmax(pred, dim=2)
    
    top_p, indxs = torch.topk(probs, k)
    top_p, indxs = top_p.squeeze().detach().numpy(), indxs.squeeze().detach().numpy()
    
    next_indx = np.random.choice(indxs, p=top_p/np.sum(top_p))
    next_char = vocab_data['i2s'][next_indx]
    next_char_prob = top_p[np.where(indxs==next_indx)].item()
    
    return next_char, next_char_prob, h

In [36]:
def sample(model, first_char='<', k=5):
    h = model.init_hidden(1)
    word = []
    
    for i in range(len(first_char)):
        char = first_char[i]
        word.append((char, 1.))
        _, char_prob, h = predict_next(char, h, k)
    
    char = first_char[-1]
    while True:
        char, char_prob, h = predict_next(char, h, k)
        word.append((char, char_prob))
        if word[-1][0] == '>':
            break

    new_word, probs, w_p = '', '', ''
    for i in word:
        new_word += str(i[0])
        probs += '{:.3f} '.format(i[1])
        w_p += '{}({:.3f}), '.format(i[0], i[1])
        
    print(new_word)
    print(w_p)

In [37]:
for _ in range(5):
    sample(model, k=5)

<каричный>
<(1.000), к(0.075), а(0.302), р(0.274), и(0.139), ч(0.014), н(0.790), ы(0.981), й(0.997), >(1.000), 
<оргенчка>
<(1.000), о(0.240), р(0.084), г(0.081), е(0.181), н(0.394), ч(0.453), к(0.077), а(0.924), >(0.999), 
<темить>
<(1.000), т(0.168), е(0.230), м(0.120), и(0.205), т(0.536), ь(0.926), >(0.943), 
<ордуюгизм>
<(1.000), о(0.126), р(0.061), д(0.204), у(0.205), ю(0.189), г(0.191), и(0.425), з(0.123), м(0.266), >(0.692), 
<еспетый>
<(1.000), е(0.109), с(0.187), п(0.184), е(0.239), т(0.140), ы(0.062), й(0.821), >(1.000), 


In [38]:
for _ in range(5):
    sample(model, k=20)

<молюнковый>
<(1.000), м(0.020), о(0.074), л(0.131), ю(0.002), н(0.055), к(0.520), о(0.038), в(0.672), ы(0.847), й(0.991), >(0.999), 
<тлата>
<(1.000), т(0.078), л(0.007), а(0.321), т(0.122), а(0.222), >(0.723), 
<зазатный>
<(1.000), з(0.037), а(0.456), з(0.006), а(0.309), т(0.055), н(0.041), ы(0.455), й(0.977), >(1.000), 
<операть>
<(1.000), о(0.220), п(0.066), е(0.081), р(0.341), а(0.626), т(0.970), ь(0.148), >(0.975), 
<убенздиетем>
<(1.000), у(0.056), б(0.214), е(0.466), н(0.074), з(0.018), д(0.106), и(0.371), е(0.126), т(0.063), е(0.136), м(0.079), >(0.104), 


In [39]:
for _ in range(5):
    sample(model, first_char='<па')

<палатина>
<(1.000), п(1.000), а(1.000), л(0.121), а(0.214), т(0.027), и(0.390), н(0.067), а(0.279), >(0.784), 
<пауский>
<(1.000), п(1.000), а(1.000), у(0.049), с(0.074), к(0.207), и(0.828), й(0.914), >(0.996), 
<палин>
<(1.000), п(1.000), а(1.000), л(0.274), и(0.497), н(0.195), >(0.283), 
<парунный>
<(1.000), п(1.000), а(1.000), р(0.133), у(0.149), н(0.443), н(0.067), ы(0.941), й(1.000), >(1.000), 
<палистический>
<(1.000), п(1.000), а(1.000), л(0.321), и(0.448), с(0.362), т(0.629), и(0.588), ч(0.626), е(0.754), с(0.905), к(0.999), и(0.990), й(0.993), >(1.000), 


In [43]:
for s in vocab_data['vocab'][4:]:
    print('Beggining is <{}'.format(s))
    for _ in range(2):
        sample(model, '<' + s, k=2)
    print()

Beggining is <а
<арис>
<(1.000), а(1.000), р(0.213), и(0.287), с(0.375), >(0.102), 
<атристрат>
<(1.000), а(1.000), т(0.226), р(0.219), и(0.330), с(0.210), т(0.725), р(0.233), а(0.684), т(0.139), >(0.119), 

Beggining is <б
<ботиться>
<(1.000), б(1.000), о(0.311), т(0.136), и(0.230), т(0.160), ь(0.942), с(0.101), я(0.996), >(0.999), 
<баватор>
<(1.000), б(1.000), а(0.285), в(0.209), а(0.397), т(0.213), о(0.462), р(0.497), >(0.824), 

Beggining is <в
<вестный>
<(1.000), в(1.000), е(0.338), с(0.289), т(0.549), н(0.123), ы(0.577), й(0.993), >(1.000), 
<важить>
<(1.000), в(1.000), а(0.269), ж(0.352), и(0.421), т(0.805), ь(0.956), >(0.935), 

Beggining is <г
<готический>
<(1.000), г(1.000), о(0.444), т(0.153), и(0.279), ч(0.376), е(0.823), с(0.997), к(0.999), и(0.993), й(0.999), >(1.000), 
<ганит>
<(1.000), г(1.000), а(0.257), н(0.361), и(0.193), т(0.563), >(0.334), 

Beggining is <д
<дига>
<(1.000), д(1.000), и(0.166), г(0.244), а(0.211), >(0.228), 
<докура>
<(1.000), д(1.000), о(0.487), к